In [5]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np
import pandas as pd
import h5py

In [6]:
with open('/home/jmanos/notebooks/Non_filtered_rhone_v2/all_rms_nofilt_1file.pkl', 'rb') as f:
    data = pickle.load(f)

data_continuous = []    

for i, time_period in enumerate(data):
    
    if i > 21565:
        data_continuous.append(time_period[0])

with open('strain_rate_times.pkl', 'rb') as f:
    strain_times = pickle.load(f)

    
strain_times_continuous = []

for i, times_strain in enumerate(strain_times):
    if i > 21565:
        strain_times_continuous.append(times_strain)
        
with open('discharge_values.pkl', 'rb') as f:
    discharge_values = pickle.load(f)

with open('discharge_times.pkl', 'rb') as f:
    discharge_times = pickle.load(f)

dt_object_dis = [datetime.strptime(time, "%Y-%m-%dT%H:%M:%SZ") for time in discharge_times]
dates_dis = mdates.date2num(dt_object_dis)

dt_object_strain = [datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ") for date in strain_times_continuous]
dates_strain = mdates.date2num(dt_object_strain)

data_continuous_formatted = np.array(data_continuous).T

discharge_values_at_strain_times = []


for strain_time in dates_strain:
    
    interp_dis = np.interp(strain_time, dates_dis, discharge_values)
    discharge_values_at_strain_times.append(interp_dis)


In [7]:
hf = h5py.File('Rhone_data_continuous_nofilt.h5', 'w')

hf.create_dataset('Times', data=strain_times_continuous)
hf.create_dataset('DAS Data', data=data_continuous)
hf.create_dataset('Discharge', data=discharge_values_at_strain_times)

hf.close()

In [10]:
#read check
hf = h5py.File('Rhone_data_continuous_nofilt.h5', 'r')
hf.keys()


<KeysViewHDF5 ['DAS Data', 'Discharge', 'Times']>

In [11]:
hf.close()